# Chapter 4, Higher-Order Linear Differential Equations

## 4.1 General Theory of nth Order Linear Differential Equations

Analytical한 증명입니다. skip하겠습니다

## 4.2 Homogeneous Differential Equations with Constant Coefficients & 4.3 The Methods of Undetermined Coefficents

앞 장에서 다루었던 내용의 확장입니다.

여기서 특성방정식을 이용한 Basis를 만드는 프로그램을 생각해보았는데, 해를 찾긴 하는데 중근을 분류해 낼 방법이 없어서 인수분해 후 중근을 직접 count하는 방법을 했습니다.

예제를 풀어보겠습니다.

In [55]:
import sympy as sy
from sympy import init_printing

init_printing(use_latex='mathjax')
t = sy.symbols('t')
f = sy.Function('f')

def make_wronskian(basis):
    row = len(basis)
    matrix = []  # 빈 행렬 생성
    
    for i in range(row):
        basis_row = [basis[i]]  # 각 기저 함수를 담은 리스트 생성
        
        for j in range(1, row):
            basis_row.append(sy.diff(basis[i], t, j))  # 각 기저 함수의 미분을 행렬에 추가
        matrix.append(basis_row)  # 각 기저 함수 행을 행렬에 추가
    
    wr = sy.Matrix(matrix).det()
    print(wr)
    
    return wr.simplify()

a = 1
b = 0
c = -4
d = 0

eq = sy.Eq(a*sy.diff(f(t), t, 3) + b * sy.diff(f(t), t, 2) +  c * sy.diff(f(t), t, 1) + d*f(t),t+ 3*sy.cos(t) +sy.exp(-2*t))
characteristic_eq = a*t**3+b*t**2+c*t+d*t

factors_list = sy.factor_list(characteristic_eq)[1]

basis = []

for factor in factors_list:
    if factor[1] != 1:
        solution = sy.solve(factor[0], t)
        base = sy.exp(solution[0]*t)
        basis.append(base)
        
        for i in range(factor[1]-1):
            base = base*t
            basis.append(base)
        
    else: 
        solutions = sy.solve(factor[0], t) 
        coeff_1 = abs(solutions[0].as_real_imag()[1])
        coeff_2 = solutions[0].as_real_imag()[0]
        if coeff_1 != 0: #허수부가 있을 경우
            if coeff_2 != 0:
                basis1 = sy.exp(coeff_2*t)* sy.cos(coeff_1*t)
                basis2 = sy.exp(coeff_2*t)* sy.sin(coeff_1*t)
            else:
                basis1 = sy.cos(coeff_1*t)
                basis2 = sy.sin(coeff_1*t)
            basis.append(basis1)
            basis.append(basis2)        
        else: #허수부가 없을 경우
            solutions = sy.solve(factor[0], t) 
            basis1 = sy.exp(solutions[0] * t)
            basis.append(basis1)
        
Wronskian = make_wronskian(basis)

if Wronskian == 0:
    print("invalid!")
else:
    print("Wronskian : ")
    display(Wronskian)

    sol = sy.dsolve(eq, f(t))

    print("solution : ", end="")
    display(sol.rhs)


-16
Wronskian : 


-16

solution : 

                2                            
         2⋅t   t    ⎛     t⎞  -2⋅t   3⋅sin(t)
C₁ + C₃⋅ℯ    - ── + ⎜C₂ + ─⎟⋅ℯ     - ────────
               8    ⎝     8⎠            5    

## 4.4 The Method of Variation of Parameters

이 부분에서는

$$
L[y] = y^(n) + p_1(t)y^(n-1)+ ... + p_n-1(t)y' + p_n(t)y = g(t)
$$일때, 

$$
L[y] = y^(n) + p_1(t)y^(n-1)+ ... + p_n-1(t)y' + p_n(t)y = g(t)
$$가 $$y_1, y_2, ... y_n$$를 fundermental set of solutions로 가지고,

$$W_m$$이 mth column을 $$(0, 0, 0, ... 1)^T$$로 대체된 matrixd의 determinant라고 하면

그 해는
$$Y(t) = \sum_{m=1}^{n} y_m(t) \left( \int_{t_0}^{t} \frac{g(s)W_m(s)}{W(s)} \, ds \right)$$
를 가집니다. 

(where t_0 is any conveniently chosen point in open interval I)

이 부분을 프로그래밍으로 해를 구하고, Variation of Parameters법으로 직접 해를 구해서 확인해보겠습니다.

In [70]:
import sympy as sy
from sympy import init_printing

init_printing(use_latex='mathjax')
t = sy.symbols('t')
f = sy.Function('f')
g = sy.Function('g')(t)
g =t

def make_wronskian(basis):
    row = len(basis)
    matrix = []  # 빈 행렬 생성
    
    basis_row = [basis[i] for i in range(row)]  # 각 기저 함수를 담은 리스트 생성
    matrix.append(basis_row)
    
    for j in range(1, row):
        basis_row = [sy.diff(base, t, j) for base in basis] # 각 기저 함수의 미분을 행렬에 추가
        matrix.append(basis_row)  # 각 기저 함수 행을 행렬에 추가
 
    wr = sy.Matrix(matrix).det()
    
    return wr.simplify()

def W_m(col_number):
    vector = [0] * len(basis)
    vector[-1] = 1
    row = len(basis)
    matrix = []  # 빈 행렬 생성
    
    basis_row = [basis[i] for i in range(row)]  # 각 기저 함수를 담은 리스트 생성
    matrix.append(basis_row)
    
    for j in range(1, row):
        basis_row = [sy.diff(base, t, j) for base in basis] # 각 기저 함수의 미분을 행렬에 추가
        matrix.append(basis_row)  # 각 기저 함수 행을 행렬에 추가
    
    for i in range(row):
        matrix[i][col_number] = vector[i]
    
    wr = sy.Matrix(matrix).det()
    
    return wr

def find_solution(basis, Wronskian):
    sol = 0
    
    for i in range(len(basis)):
        sol += basis[i]*sy.integrate(g* W_m(i)/Wronskian, t)
    
    return sol


a = 1
b = 0
c = -1
d = 0


eq = sy.Eq(a*sy.diff(f(t), t, 3) + b * sy.diff(f(t), t, 2) +  c * sy.diff(f(t), t, 1) + d*f(t),g)
characteristic_eq = a*t**3+b*t**2+c*t+d*t

factors_list = sy.factor_list(characteristic_eq)[1]

basis = []

for factor in factors_list:
    if factor[1] != 1:
        solution = sy.solve(factor[0], t)
        base = sy.exp(solution[0]*t)
        basis.append(base)
        
        for i in range(factor[1]-1):
            base = base*t
            basis.append(base)
        
    else: 
        solutions = sy.solve(factor[0], t) 
        coeff_1 = abs(solutions[0].as_real_imag()[1])
        coeff_2 = solutions[0].as_real_imag()[0]
        if coeff_1 != 0: #허수부가 있을 경우
            if coeff_2 != 0:
                basis1 = sy.exp(coeff_2*t)* sy.cos(coeff_1*t)
                basis2 = sy.exp(coeff_2*t)* sy.sin(coeff_1*t)
            else:
                basis1 = sy.cos(coeff_1*t)
                basis2 = sy.sin(coeff_1*t)
            basis.append(basis1)
            basis.append(basis2)        
        else: #허수부가 없을 경우
            solutions = sy.solve(factor[0], t) 
            basis1 = sy.exp(solutions[0] * t)
            basis.append(basis1)
        
Wronskian = make_wronskian(basis)

if Wronskian == 0:
    print("invalid!")
else:
    print("Wronskian : ")
    display(Wronskian)

    sol = sy.dsolve(eq, f(t))

    print("solution by programming: ")
    display(sol.rhs)
    
    print("solution by Variation of Parameters: ")
    display(find_solution(basis, Wronskian))
    

Wronskian : 


-2

solution by programming: 


                       2
         -t       t   t 
C₁ + C₂⋅ℯ   + C₃⋅ℯ  - ──
                      2 

solution by Variation of Parameters: 


   2    
  t     
- ── - 1
  2     

C_1을 -1이 되게끔 t_0를 설정하면 양쪽 답이 같게 나옴을 알 수 있었습니다.